In [ ]:
# |default_exp utils

In [ ]:
# |export
import logging
from datetime import datetime
from threading import Lock, Thread
from time import sleep, time

import psutil

# Utils
> Various utilities for trackmania-ai project

In [ ]:
# |export
def log_memory(suffix=""):
    logging.debug(
        f"Current process uses {psutil.Process().memory_info().rss / (1024 * 1024)}MB{suffix}"
    )

In [ ]:
# |export
def format_date(f="%Y/%m/%d %H:%M"):
    return datetime.now().strftime(f)


def format_duration(seconds):
    if not seconds:
        return "0s"
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    d, h = divmod(h, 24)
    return "".join(f"{v}{l}" for v, l in zip([d, h, m, s], list("dhms")) if v)

In [ ]:
# |export
class IntervalCall(Thread):
    def __init__(self, fn, interval):
        super().__init__()
        self.fn, self.interval = fn, interval
        self.should_stop = False
        self.start()

    def run(self):
        while not self.should_stop:
            self.fn()
            sleep(self.interval)

    def __del__(self):
        self.should_stop = True
        self.join

In [ ]:
# |export
class Counter:
    def __init__(self, value=0):
        self.val = value
        self._lock = Lock()

    def increment(self):
        with self._lock:
            self.val += 1

    def value(self):
        with self._lock:
            return self.val

In [ ]:
# |export
class Timer:
    def __init__(self, timeout):
        self.start_time = time()
        self.timeout = timeout

    def __call__(self):
        return self.timeout < time() - self.start_time

    def progress(self):
        return min((time() - self.start_time) / self.timeout, 1.0)

    def reinit(self):
        self.start_time = time()

In [ ]:
# |export
class Triggers:
    def __init__(self, trigger_values, condition=lambda trig, val: trig < val):
        self.trigger_values, self.condition = trigger_values, condition
        self.reset()

    def reset(self):
        self.done = False
        self.gen = (x for x in self.trigger_values)
        self._next()

    def _next(self):
        try:
            self.trigger = next(self.gen)
        except StopIteration:
            self.trigger = None
            self.done = True

    def __call__(self, value):
        if self.done:
            return False
        if self.condition(self.trigger, value):
            self._next()
            return True
        return False

In [ ]:
# |export
def attrsToDict(obj, attrs):
    return {attr: getattr(obj, attr) for attr in attrs}

In [ ]:
# |hide
# |eval: false
from nbdev.doclinks import nbdev_export

nbdev_export()